### Deliverable 3: Optimize the Model

In [21]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from keras.callbacks import Callback, ModelCheckpoint
import pandas as pd
import tensorflow as tf
import os

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Preprocess the Dataset

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine which values to replace 
types = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(types[types < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
classes = application_df['CLASSIFICATION'].value_counts()
# Determine which values to replace
replace_class = list(classes[classes < 1880].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\ajdea\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
merged_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(columns=application_cat)
merged_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Design a Neural Network Model

In [7]:
# Split our preprocessed data into our features and target arrays
y = merged_df['IS_SUCCESSFUL'].values
X = merged_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\ajdea\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


# Optimization Attempt 1: Additional Layers and Neurons

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               5280      
                                                                 
 dense_1 (Dense)             (None, 80)                9680      
                                                                 
 dense_2 (Dense)             (None, 30)                2430      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 17,421
Trainable params: 17,421
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

#Create a callback that saves the model's weight every 5 epochs
callback_5 = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

# Train the model
fit_model = nn.fit(X_train,y_train,callbacks=[callback_5],epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 627us/step - loss: 0.6921 - accuracy: 0.5255
Epoch 2/100
804/804 [==============================] - 1s 622us/step - loss: 0.6919 - accuracy: 0.5290
Epoch 3/100
804/804 [==============================] - 1s 628us/step - loss: 0.6921 - accuracy: 0.5303
Epoch 4/100
804/804 [==============================] - 0s 617us/step - loss: 0.6921 - accuracy: 0.5268
Epoch 5/100
730/804 [==========================>...] - ETA: 0s - loss: 0.6920 - accuracy: 0.5265
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 633us/step - loss: 0.6920 - accuracy: 0.5272
Epoch 6/100
804/804 [==============================] - 0s 617us/step - loss: 0.6924 - accuracy: 0.5268
Epoch 7/100
804/804 [==============================] - 1s 627us/step - loss: 0.6921 - accuracy: 0.5252
Epoch 8/100
804/804 [==============================] - 0s 618us/step - loss: 0.6920 - accuracy: 0.5262
Epoch 9/100
804/804 [=============

804/804 [==============================] - 1s 637us/step - loss: 0.6919 - accuracy: 0.5268
Epoch 64/100
804/804 [==============================] - 1s 628us/step - loss: 0.6918 - accuracy: 0.5269
Epoch 65/100
485/804 [=================>............] - ETA: 0s - loss: 0.6918 - accuracy: 0.5257
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 632us/step - loss: 0.6921 - accuracy: 0.5251
Epoch 66/100
804/804 [==============================] - 1s 632us/step - loss: 0.6920 - accuracy: 0.5276
Epoch 67/100
804/804 [==============================] - 1s 627us/step - loss: 0.6923 - accuracy: 0.5285
Epoch 68/100
804/804 [==============================] - 1s 622us/step - loss: 0.6921 - accuracy: 0.5268
Epoch 69/100
804/804 [==============================] - 0s 617us/step - loss: 0.6921 - accuracy: 0.5253
Epoch 70/100
482/804 [================>.............] - ETA: 0s - loss: 0.6924 - accuracy: 0.5259
Epoch 00070: saving model to checkpoints\weig

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7049 - accuracy: 0.5426 - 165ms/epoch - 616us/step
Loss: 0.7049376368522644, Accuracy: 0.5426238775253296


# Optimization Attempt 2: Change Activation of Layers

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 95
hidden_nodes_layer2 = 42

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 95)                4180      
                                                                 
 dense_11 (Dense)            (None, 42)                4032      
                                                                 
 dense_12 (Dense)            (None, 1)                 43        
                                                                 
Total params: 8,255
Trainable params: 8,255
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,callbacks=[callback_5],epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 630us/step - loss: 0.6923 - accuracy: 0.5255
Epoch 2/100
804/804 [==============================] - 1s 635us/step - loss: 0.6922 - accuracy: 0.5288
Epoch 3/100
804/804 [==============================] - 1s 638us/step - loss: 0.6927 - accuracy: 0.5241
Epoch 4/100
782/804 [============================>.] - ETA: 0s - loss: 0.6923 - accuracy: 0.5253
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 655us/step - loss: 0.6924 - accuracy: 0.5244
Epoch 5/100
804/804 [==============================] - 1s 639us/step - loss: 0.6924 - accuracy: 0.5268
Epoch 6/100
804/804 [==============================] - 1s 643us/step - loss: 0.6924 - accuracy: 0.5228
Epoch 7/100
804/804 [==============================] - 1s 634us/step - loss: 0.6926 - accuracy: 0.5286
Epoch 8/100
804/804 [==============================] - 1s 640us/step - loss: 0.6924 - accuracy: 0.5276
Epoch 9/100
692/804 [=============

804/804 [==============================] - 1s 635us/step - loss: 0.6922 - accuracy: 0.5243
Epoch 64/100
547/804 [===================>..........] - ETA: 0s - loss: 0.6934 - accuracy: 0.5178
Epoch 00064: saving model to checkpoints\weights.64.hdf5
804/804 [==============================] - 1s 649us/step - loss: 0.6925 - accuracy: 0.5236
Epoch 65/100
804/804 [==============================] - 1s 643us/step - loss: 0.6922 - accuracy: 0.5269
Epoch 66/100
804/804 [==============================] - 1s 666us/step - loss: 0.6926 - accuracy: 0.5224
Epoch 67/100
804/804 [==============================] - 1s 650us/step - loss: 0.6923 - accuracy: 0.5253
Epoch 68/100
804/804 [==============================] - 1s 655us/step - loss: 0.6923 - accuracy: 0.5274
Epoch 69/100
464/804 [================>.............] - ETA: 0s - loss: 0.6924 - accuracy: 0.5273
Epoch 00069: saving model to checkpoints\weights.69.hdf5
804/804 [==============================] - 1s 653us/step - loss: 0.6923 - accuracy: 0.5264
E

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8650 - accuracy: 0.5348 - 167ms/epoch - 623us/step
Loss: 0.8649754524230957, Accuracy: 0.534810483455658


# Optimization Attempt 3: Remove Noisy Variables

In [ ]:
# Drop more columns

In [34]:
merged_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [36]:
trim_df = merged_df.drop(columns=['ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y', 
        'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
        'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional'])
trim_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Split our trimmed data into our features and target arrays
y = trim_df['IS_SUCCESSFUL'].values
X = trim_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\ajdea\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 80)                2160      
                                                                 
 dense_14 (Dense)            (None, 30)                2430      
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4,621
Trainable params: 4,621
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train,y_train,callbacks=[callback_5],epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 599us/step - loss: 54040.2461 - accuracy: 0.4865
Epoch 2/100
804/804 [==============================] - 0s 604us/step - loss: 73061.4766 - accuracy: 0.4834
Epoch 3/100
804/804 [==============================] - 0s 598us/step - loss: 73969.0781 - accuracy: 0.5027
Epoch 4/100
333/804 [===========>..................] - ETA: 0s - loss: 7237.5669 - accuracy: 0.4916
Epoch 00004: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 0s 618us/step - loss: 38021.8125 - accuracy: 0.5059
Epoch 5/100
804/804 [==============================] - 0s 602us/step - loss: 48229.7227 - accuracy: 0.4907
Epoch 6/100
804/804 [==============================] - 0s 599us/step - loss: 23927.3984 - accuracy: 0.4913
Epoch 7/100
804/804 [==============================] - 0s 597us/step - loss: 44156.5312 - accuracy: 0.4900
Epoch 8/100
804/804 [==============================] - 0s 597us/step - loss: 26386.3066 - accuracy: 0.4918

804/804 [==============================] - 0s 598us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 63/100
804/804 [==============================] - 0s 603us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 64/100
 85/804 [==>...........................] - ETA: 0s - loss: 0.6895 - accuracy: 0.5445
Epoch 00064: saving model to checkpoints\weights.64.hdf5
804/804 [==============================] - 0s 610us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 65/100
804/804 [==============================] - 0s 600us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 66/100
804/804 [==============================] - 0s 597us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 67/100
804/804 [==============================] - 0s 609us/step - loss: 0.6915 - accuracy: 0.5318
Epoch 68/100
804/804 [==============================] - 0s 599us/step - loss: 0.6913 - accuracy: 0.5304
Epoch 69/100
 83/804 [==>...........................] - ETA: 0s - loss: 0.6904 - accuracy: 0.5395
Epoch 00069: saving model to checkpoints\weig

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.2189 - accuracy: 0.5472 - 164ms/epoch - 612us/step
Loss: 1.2188869714736938, Accuracy: 0.5471720099449158


# Export Results

In [41]:
# Save and export results to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")